In [3]:
stocks = {'BNP Paribas': 'Bank', 'Carrefour': 'Food', 'Cement Roadstone Hold': 'Construction', 
          'Daimler': 'Automobiles', 'Danone': 'Food', 'Deutsche Bank': 'Bank', 'Deutsche Telekom': 'Telecommunications',
          'ENEL': 'Electricity', 'Engie': 'Electricity', 'ENI': 'Oil & Gas', 'EON': 'Electricity',
          'Essilor Intl': 'Health Care', 'Iberdrola': 'Electricity', 'Industria Textil': 'Personal Goods',
          'ING Group': 'Bank', 'Intesa SP': 'Bank', 'L’Oreal': 'Personal Goods', 'LVMH': 'Leisure Goods',
          'Munich RE': 'Insurance', 'Orange': 'Telecommunications', 'Philips': 'Electrical Equipment',
          'Repsol': 'Oil & Gas', 'RWE AG': 'Electricity', 'Saint Gobain': 'Construction', 'Sanofi': 'Pharmaceuticals',
          'SAP': 'Software', 'Schneider': 'Hardware', 'Siemens': 'Electrical Equipment', 'Societe Generale': 'Bank',
          'Telefonica SA': 'Telecommunications', 'Total': 'Oil & Gas', 'UniCredit': 'Bank',
          'Unilever': 'Household Goods', 'Vinci': 'Constructions', 'Vivendi': 'Telecommunications', 
          'Volkswagen': 'Automobiles', 'Air Liquide': 'Chemicals', 'Allianz SE': 'Insurance',
          'ABI': 'Food', 'Arcelor Mittal': 'Construction', 'ASML': 'Electrical Equipments',
          'Generali': 'Insurance', 'AXA': 'Insurance', 'BBVA': 'Bank', 'Santander': 'Bank', 'BASF': 'Chemicals',
          'Bayer': 'Pharmaceuticals', 'BMW': 'Automobiles'
         }